In [ ]:
#general 
import pandas as pd
import numpy as np
import os
from sklearn import __version__ as sklearn_version
from sklearn.model_selection import train_test_split

#warnings 
import warnings 
warnings.filterwarnings("ignore")

#for RNN
import tensorflow as tf
from tensorflow.keras.callbacks import EarlyStopping
from keras.models import load_model

import pickle

import ast

Create Sentences for testing RNNS

In [ ]:
year = "2012_RNN"
path = "C:/Users/danie/Desktop/Masters Thesis/RNN Models Colab/"

In [ ]:
word_df = pd.read_pickle(r"C:\Users\danie\Documents\GitHub\Masters-Thesis\Word Databases\pos_tag_word_list")
sent_df = pd.read_csv(r"C:\Users\danie\Documents\GitHub\Masters-Thesis\RNN Sentences\sample sent idea.csv")

Develop Benchmarks

In [ ]:
def benchmark_string(sents):
    list_of_strings = []

    for word_list in sents:
        list_of_strings.append(' '.join(word_list))
    return list_of_strings

In [ ]:
sent_df['list'] = sent_df['list'].apply(ast.literal_eval)
benchmark_sentences = sent_df['list'].tolist()
all_benchmarks = benchmark_string(benchmark_sentences)

In [ ]:
loaded_model = tf.keras.models.load_model(path+year)

In [ ]:
X_b = all_benchmarks
x_arrayb = np.asarray(X_b)
y_predb = loaded_model.predict(X_b)
Yb = pd.DataFrame(y_predb)
Yb.reset_index(inplace=True, drop=True)
Yb.set_axis(['0 Class Benchamark', '1 Class Benchmark'], axis='columns', inplace=True)
sent_df = pd.concat([sent_df, Yb], axis=1)

Test RNN model

#NOUNS

In [ ]:
sent_df['replacement string'] = sent_df['replacement string'].apply(ast.literal_eval)


noun_list = word_df[(word_df.pos_tag_short == "N")]["feature"].values.tolist()
verb_list = word_df[(word_df.pos_tag_short == "V")]["feature"].values.tolist()
#adverb_list = word_df[(word_df.pos_tag_short == "R")]["feature"].values.tolist()
adjective_list = word_df[(word_df.pos_tag_short == "J")]["feature"].values.tolist()

noun_sent = sent_df[(sent_df['POS tag'] == "Noun")]["replacement string"].values.tolist()
verb_sent = sent_df[(sent_df['POS tag'] == "Verb")]["replacement string"].values.tolist()
adjective_sent = sent_df[(sent_df['POS tag'] == "Adjective")]["replacement string"].values.tolist()
#adverb_sent = sent_df[(sent_df['POS tag'] == "Adverb")]["replacement string"].values.tolist()

#adverb_len = len(adverb_sent)
adjective_len = len(adjective_sent)
noun_len = len(noun_sent)
verb_len = len(verb_sent)

In [ ]:
def word_list(sublist, sentences):
    output=[]
    for sentence in sentences:
        index=sentence.index('UNK')
        sentence_copy = sentence.copy()
        for sub in sublist:
            sentence_copy[index]=sub
            output.append([' '.join(sentence_copy.copy()),sentence])

    return output



sent_n = pd.DataFrame(word_list(noun_list, noun_sent))
sent_j = pd.DataFrame(word_list(adjective_list, adjective_sent))
#sent_a = pd.DataFrame(word_list(adverb_list, adverb_sent))
sent_v = pd.DataFrame(word_list(verb_list, verb_sent))

In [ ]:
x_array_n = np.asarray(sent_n[0])
x_array_j = np.asarray(sent_j[0])
x_array_v = np.asarray(sent_v[0])
#x_array_a = np.asarray(sent_a[0])


y_pred_n = loaded_model.predict(x_array_n)
y_pred_j = loaded_model.predict(x_array_j)
y_pred_v = loaded_model.predict(x_array_v)
#y_pred_a = loaded_model.predict(x_array_a)

In [ ]:
#You need to change the num_repititons to the number of sentences in the function

def labels(word_list, word_len):
    num_repetitions = word_len
    repeated_words = []
    repeated_words = np.tile(word_list, word_len)
    return repeated_words

noun_labels = pd.DataFrame(labels(noun_list, noun_len))
adjective_labels = pd.DataFrame(labels(adjective_list, adjective_len))
verb_labels = pd.DataFrame(labels(verb_list, verb_len))
#adverb_labels = pd.DataFrame(labels(adverb_list, adverb_len))



In [ ]:
Y_n = pd.DataFrame(y_pred_n)
Y_j = pd.DataFrame(y_pred_j)
Y_v = pd.DataFrame(y_pred_v)
#Y_a = pd.DataFrame(y_pred_a)

result_n = pd.concat([noun_labels, Y_n,sent_n], axis=1)
result_j = pd.concat([adjective_labels, Y_j,sent_j], axis=1)
result_v = pd.concat([verb_labels, Y_v,sent_v], axis=1)
#result_a = pd.concat([adverb_labels, Y_a, sent_a], axis=1)

result_n.set_axis(['feature', '0 Class', '1 Class', 'RNN Sentences', 'replacement string'], axis='columns', inplace=True)
result_j.set_axis(['feature', '0 Class', '1 Class', 'RNN Sentences', 'replacement string'], axis='columns', inplace=True)
result_v.set_axis(['feature', '0 Class', '1 Class', 'RNN Sentences', 'replacement string'], axis='columns', inplace=True)
#result_a.set_axis(['feature', '0 Class', '1 Class', 'RNN Sentences', 'replacement string'], axis='columns', inplace=True)

In [ ]:
result = pd.concat([result_v, result_j, result_n])

In [ ]:
result_with_source = pd.merge(result, word_df, on='feature', how='left')
#verb problem

In [ ]:
result_with_source = result_with_source.dropna()
result_with_source['replacement string'] = result_with_source['replacement string'].apply(tuple)
sent_df['replacement string'] = sent_df['replacement string'].apply(tuple)

In [ ]:
df = pd.merge(result_with_source, sent_df, on='replacement string', how='inner')

In [ ]:
df['Class 0 Differential'] = df['0 Class'] - df['0 Class Benchamark']
df['Class 1 Differential'] = df['1 Class'] - df['1 Class Benchmark']


In [ ]:
#add a section that labels the columns for the year and drops the uneeded oned. 
df = df.drop(['list','sent','pos_tag','pos_tag_short','replacement string', 'word num','identified word'], axis= 1)

In [ ]:
df = df.set_axis( ['feature',year + '_0_class', year + '_1_Class','RNN Sentences', 'source', 'sent_id','POS tag', year + '_0_Class_Benchamark',year + '_1_Class_Benchmark', year + '_Class_0_Differential', year + '_Class_1_Differential'], axis = 'columns')

In [ ]:
file_name = "RNN_Test_Sentences_"+year

with open(file_name, 'wb') as handle:
    pickle.dump(df, handle, protocol=pickle.HIGHEST_PROTOCOL)